In [1]:
from source.regency import Regency
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')


Game = Regency()
clear_output()
for a in range(1):
    Game.random_events()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.collect_regency_points()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.domain_initiative()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.collect_gold_bars()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.maintenance_costs()
    print(len((Game.Seasons[Game.Season]['Season'])))

Game.clear_screen()
Game.Seasons[Game.Season]['Season']

,Regent,Player,Random Event,Regency Points,Initiative,Gold Bars,Revenue,Cost,Court
0,EC,False,No Event,33,20,-1,4,5,Dormant
1,Sw2,False,No Event,24,20,1,0,2,Bare
2,ML,False,Brigandage,3,19,0,3,3,Bare
3,EL,False,No Event,12,17,5,8,3,Bare
4,LPA,False,Festival,11,16,7,11,4,Bare
5,TCV,False,No Event,1,14,0,1,1,Dormant
6,TF,False,No Event,2,14,1,2,1,Dormant
7,Is,False,No Event,106,14,0,7,7,Bare
8,WM,False,Assassination,7,13,1,2,1,Dormant
9,JR,False,No Event,33,13,-4,-1,3,Dormant


In [2]:
Game.take_domain_actions(None)

In [3]:
Game.Troops

,Regent,Provence,Type,Cost,CR,Garrisoned,Home
0,EC,Bogsend,Pikemen,1,1.0,0,
1,GG,Rivenrock,Dwarf Guards,2,2.0,0,
2,GG,Rivenrock,Dwarf Crossbows,2,2.0,0,
3,GG,Rivenrock,Dwarf Guards,2,2.0,0,
4,GG,Rivenrock,Dwarf Crossbows,2,2.0,0,
5,GG,Rivenrock,Dwarf Guards,2,2.0,0,
6,GG,Rivenrock,Dwarf Crossbows,2,2.0,0,
7,GG,Rivenrock,Dwarf Guards,2,2.0,0,
8,GG,Rivenrock,Dwarf Crossbows,2,2.0,0,
9,CoL,Salviene,Scouts,1,0.5,1,
